# Maximum Likelihood Estimators and SymPy

A discrete random variable $X$  has range $\{1,2,3,4\}$  and probability mass function

$$
p(x; \theta) =
\begin{cases}
  \theta \hspace{2mm} & \text{for } x=1; \\
  \frac{1}{2}(1-\theta) \hspace{2mm} & \text{for } x=2; \\
  \frac{1}{4}(1-\theta) \hspace{2mm} & \text{for } x=3; \\
  \frac{1}{4}(1-\theta) \hspace{2mm} & \text{for } x=4.
\end{cases}
$$

The parameter $\theta \in (0,1)$  is unknown.

A random sample of 100 independent observations were obtained from which to estimate $\theta$: these data are given in the following frequency table.

| $x$           | 1    | 2    | 3    | 4    |
| ------------- | ---- | ---- | ---- | ---- |
| **Frequency** | 31   | 37   | 18   | 14   |

What is the likelihood $L(\theta)$ for these data?

In [7]:
from sympy import diff, solve, lambdify, Float, solve
from sympy.abc import theta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Let us first define `np.arrays` for

- $x$;
- $p(x; \theta)$;
- $f_{x}$.

In [3]:
x = np.array([1, 2, 3, 4])
pmf = np.array([theta, (1/2)*(1-theta), (1/4)*(1-theta), (1/4)*(1-theta)])
f = np.array([31, 37, 18, 14])  # frequency of outcome

Declare $L(\theta)$

In [5]:
lik_gen = np.array([pmf[i]**f[i] for i in range(0, x.size)])
lik = np.prod([lik_gen])

Declare $L'(\theta)$

In [6]:
maxlik = diff(lik, theta)

Solve $L'(\theta) = 0$.

In [12]:
sol = solve(maxlik)[1]
lower = float(solve(maxlik)[0])
upper = float(solve(maxlik)[2])

print("MLE = {}".format(sol))

MLE = 0.310000000000000


In [ ]:
# =============================================================================
# lamdify pmf and likelihood
# =============================================================================

thetas = np.linspace(start=lower, stop=upper, num=1000)  # possible thetas
lik = lambdify([theta], lik.prod(), "numpy")(thetas)  # calculate li(theta)

exp = list()
k = 0  # reinitialise counter
while k < x.size:
    exp.append(lambdify([theta], f.sum()*pmf[k]))  # build list
    k +=1
exp = np.asarray(exp)  # list as np.array


# =============================================================================
# Plot li
# =============================================================================

fig, ax = plt.subplots()

sns.lineplot(x=thetas, y=lik, ax=ax)
ax.plot(sol, lik.max(), 'o', c="r",
        label='MLE = {}'.format(round(sol, 3)))  # add marker for MLE
ax.plot(np.repeat([float(sol)], 100),
        np.linspace(0, lik.max(), 100),
        '--', lw=1, c="r")  # add drop line
ax.legend(loc=0)

# =============================================================================
# Output Observed/Expected table
# =============================================================================

expected = list()
k = 0  # reinitialise counter
while k < x.size:
    expected.append(Float(exp[k](sol), 4))  # build list
    k += 1
expected = np.asarray(expected)  # list as np.array

df = pd.DataFrame(data={'X': x, 'Observed': f, 'Expected': expected})

# =============================================================================
# Plot tbl as side-by-size barplot
# =============================================================================

df_melt = pd.melt(df, 'X')

fig1, ax1 = plt.subplots()
sns.barplot(data=pd.melt(df, 'X'),
            x='X',
            y="value",
            hue='variable')

In [2]:
# =============================================================================
# define a, b as constants
# =============================================================================

a, b = symbols('a b', constants=True)

In [3]:
# =============================================================================
# define the pmf
# =============================================================================

p1 = theta**31
p2 = (a*(1 - theta))**37
p3 = ((b)*(1-theta))**18
p4 = ((b)*(1-theta))**14

In [4]:
# =============================================================================
# find L()
# =============================================================================

li = p1*p2*p3*p4
li

a**37*b**32*theta**31*(1 - theta)**69

In [5]:
# =============================================================================
# differentiate li
# =============================================================================

maxli = diff(li, theta)
maxli

-69*a**37*b**32*theta**31*(1 - theta)**68 + 31*a**37*b**32*theta**30*(1 - theta)**69

In [6]:
# =============================================================================
# Simplify maxli
# =============================================================================

maxli = simplify(maxli)
maxli

a**37*b**32*theta**30*(31 - 100*theta)*(theta - 1)**68

In [7]:
# =============================================================================
# Solve maxli=0
# =============================================================================

solve(Eq(maxli, 0))

[{a: 0}, {b: 0}, {theta: 0}, {theta: 31/100}, {theta: 1}]